In [38]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.1'
spark_version = 'spark-3.4.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 229 kB in 1s (214 kB/s)
Reading package lists... Done


In [39]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import year
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [40]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [41]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)
df.createOrReplaceTempView('home_sales')
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [42]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [43]:
df.select(year(df["date"])year(df["date"])).show()

SyntaxError: ignored

In [44]:
df = df.withColumn("year", year(df['date']))
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|2022|
|5aa00529-0533-46b...|2019-01-30|      2

In [57]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query = """
(SELECT YEAR(date) AS Year,ROUND(avg(price),2) AS Average FROM home_sales
WHERE bedrooms==4
GROUP BY YEAR(date)

)
"""
spark.sql(query).show()



+----+---------+
|Year|  Average|
+----+---------+
|2022|296363.88|
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
+----+---------+



In [58]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query = """
(SELECT YEAR(date) AS Year,ROUND(avg(price),2) AS Average FROM home_sales
WHERE bedrooms==3 AND bathrooms==3
GROUP BY YEAR(date)

)
"""
spark.sql(query).show()



+----+---------+
|Year|  Average|
+----+---------+
|2022|292725.69|
|2019|287287.82|
|2020|294204.16|
|2021|294211.46|
+----+---------+



In [67]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

query = """
(SELECT date_built ,ROUND(avg(price),2) AS Average FROM home_sales
WHERE bedrooms==3 AND bathrooms==3 AND floors==2 AND sqft_lot>=2000
GROUP BY date_built

)
"""
spark.sql(query).show()


+----------+---------+
|date_built|  Average|
+----------+---------+
|      2015|291788.36|
|      2013|295142.13|
|      2014|294195.13|
|      2012|295858.68|
|      2016| 287812.6|
|      2010|280447.23|
|      2011|281413.45|
|      2017|282026.59|
+----------+---------+



In [69]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
query = """
(SELECT view ,ROUND(avg(price),2) AS Average FROM home_sales
WHERE price>=35000
GROUP BY view

)
"""
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view|   Average|
+----+----------+
|  31| 287988.84|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 286124.07|
|  81|1053472.79|
|  28| 285474.25|
|  76|1058802.78|
|  26| 281224.95|
|  27|  281702.6|
|  44| 282856.44|
|  12| 286575.22|
|  91|1137372.73|
|  22| 284908.42|
|  93|1026006.06|
|  47| 292925.62|
|   1| 288248.11|
|  52| 733780.26|
|  13| 285849.95|
+----+----------+
only showing top 20 rows

--- 0.7591581344604492 seconds ---


In [70]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")


DataFrame[]

In [71]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [72]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query = """
(SELECT view ,ROUND(avg(price),2) AS Average FROM home_sales
WHERE price>=35000
GROUP BY view

)
"""
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view|   Average|
+----+----------+
|  31| 287988.84|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 286124.07|
|  81|1053472.79|
|  28| 285474.25|
|  76|1058802.78|
|  26| 281224.95|
|  27|  281702.6|
|  44| 282856.44|
|  12| 286575.22|
|  91|1137372.73|
|  22| 284908.42|
|  93|1026006.06|
|  47| 292925.62|
|   1| 288248.11|
|  52| 733780.26|
|  13| 285849.95|
+----+----------+
only showing top 20 rows

--- 0.5189120769500732 seconds ---


In [73]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("homesales_partitioned")

In [74]:
# 11. Read the parquet formatted data.
parquet_df_hs=spark.read.parquet('homesales_partitioned')

In [75]:
# 12. Create a temporary table for the parquet data.
parquet_df_hs.createOrReplaceTempView('p_homesales_p')

In [76]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()
query = """
(SELECT view ,ROUND(avg(price),2) AS Average FROM p_homesales_p
WHERE price>=35000
GROUP BY view

)
"""
spark.sql(query).show()



print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view|   Average|
+----+----------+
|  31| 287988.84|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 286124.07|
|  81|1053472.79|
|  28| 285474.25|
|  76|1058802.78|
|  26| 281224.95|
|  27|  281702.6|
|  44| 282856.44|
|  12| 286575.22|
|  91|1137372.73|
|  22| 284908.42|
|  93|1026006.06|
|  47| 292925.62|
|   1| 288248.11|
|  52| 733780.26|
|  13| 285849.95|
+----+----------+
only showing top 20 rows

--- 1.3776757717132568 seconds ---


In [77]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [78]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False